In [44]:
import lmfit
import numpy as np
import matplotlib.pyplot as plt
from multipletau import autocorrelate
%matplotlib inline

In [ ]:
#Load text file as a matrix, it may take a while, file has 8 million data points
M = np.loadtxt('Example_FCS_data.txt')
#Select first array as timestamps and second array as counts
t = M[:,0]
count=M[:,1]
#plot count against t
#plt.plot(t,count)


In [ ]:
#calcualte autocorrelation function
dt=t[1]-t[0]/1e9
ACF = autocorrelate(count, m=16,deltat=dt, normalize=True)

#plot
plt.figure(figsize=(12,10))
plt.semilogx(ACF[:,0], ACF[:,1], 'o')
plt.xlabel(r'$\tau (s)$',fontsize=12)
plt.ylabel(r'$G(\tau)$',fontsize=12)
plt.ylim(0,.05)

In [ ]:
def FCS_model(tau, G0, D,w, wz, G_inf):
    return G0*((1+ 4*D*tau/w**2)**-1 * (1+ 4*D*tau*wz**-2)**-0.5) + G_inf


In [ ]:
model = lmfit.Model(FCS_model)
params = model.make_params(G0=1.,D=300,w=0.290,wz=68,G_inf=0.)
params['wz'].set(value=1.7, vary=False)
params['D'].set(min=1.e-8,max=+np.inf,vary=True)
params['w'].set(value=0.342, vary=False)
ACF = np.delete(ACF,0,axis=0)
fitres = model.fit(ACF[:,1], tau=ACF[:,0], params=params, method='least_squares')
print(fitres.fit_report())


In [ ]:
plt.semilogx(ACF[:,0], ACF[:,1],'-')
plt.semilogx(ACF[:,0], fitres.best_fit)
plt.xlabel(r'$\tau (s)$')
plt.ylabel(r'$G(\tau)$')
plt.ylim(0,0.04)
plt.show()
np.savetxt("ACFandFit.txt",np.transpose([ACF[:,0],ACF[:,1],fitres.best_fit]),delimiter='\t')
# first two points are higher correlations due to triplet state blinking, discard or change equation of fit to account for it

In [ ]:
f = open('fitting report.txt','w')
f.write(fitres.fit_report())
f.close()
